In [33]:
#from database_cloudant import CloudantDB
#from cloudant.design_document import DesignDocument
def get_monthly_topwords():
    db10 = CloudantDB("monthlytopwords")
    data = db10.get_data()
    lst = []

    for month_data in data:
        month = month_data["_id"]
        for key in month_data:
            if key != "_id" and key!= "_rev":
                for word in month_data[key]:
                    count = month_data[key][word]
                    lst.append([month, key, word, count])
    df = pd.DataFrame(lst)
    df.columns = ["date", "zone", "word", "count"]
    return df

In [34]:
get_monthly_topwords()

Username: admin
Databases: ['monthlytopwords', 'topwordsmonth3', 'tweets_covid', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: monthlytopwords


,date,zone,word,count
0,2020-06,Sydney,Sydney,8
1,2020-06,Sydney,Australia,8
2,2020-06,Sydney,https://t.co/DG7OHGPvDP,6
3,2020-06,Sydney,#BlackLivesMatter,5
4,2020-06,Sydney,Check,5
...,...,...,...,...
1008,2021-05,Perth,killing,1
1009,2021-05,Perth,women,1
1010,2021-05,Perth,children,1
1011,2021-05,Perth,America,1


In [16]:
popdict

{'_id': '2021-04',
 '_rev': '1-cf0b2bd10358c18c2d8196bdbe77d4d8',
 'Adelaide': {'Adelaide': 281,
  'Australia': 203,
  'South': 163,
  'posted': 141,
  'photo': 134,
  'today': 125,
  'people': 82,
  'great': 80,
  'Happy': 68,
  'video': 68},
 'Ballarat': {'#walkies': 24,
  'Ballarat': 23,
  'Abstract': 16,
  '#zerowaste': 14,
  '#recycle': 13,
  '#MAFS': 12,
  '#abstractart': 12,
  '#artist': 11,
  '#recycling': 11,
  '#artwork': 11},
 'Brisbane': {'posted': 961,
  'photo': 938,
  'Brisbane': 489,
  'Queensland': 476,
  'Australia': 350,
  'today': 201,
  'people': 126,
  'great': 119,
  'night': 114,
  'first': 111},
 'Bunbury': {'Bunbury': 7,
  'Western': 4,
  'Australia': 4,
  'posted': 3,
  'photo': 3,
  '#housesitting': 2,
  'walking': 2,
  'today': 2,
  'Taking': 1,
  'break': 1},
 'Canberra': {'Canberra': 168,
  'posted': 101,
  'photo': 97,
  'Australian': 71,
  'Australia': 65,
  'great': 58,
  'eWasp': 58,
  'today': 55,
  'people': 52,
  'morning': 50},
 'Geelong': {'Geelo

In [15]:
import json
with open('top10wordstotal.json', 'w') as f:
    json.dump(popdict, f)

    
f.close()

In [6]:
db1=CloudantDB("tweets_no_keywords")

Username: admin
Databases: ['topwordsmonth3', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


In [13]:
len(db1.get_data(n=400000))

76526

In [22]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 19 00:40:53 2021

@author: Windwalker
"""
USERNAME='admin'
PASSWORD = 'password'
URL = 'http://172.26.131.218:5984'
from cloudant.client import CouchDB
from cloudant.design_document import DesignDocument
from cloudant.view import View

class CloudantDB():
    def __init__(self,db_name,username = USERNAME,password=PASSWORD,url = URL,partition=True):
        self.client = CouchDB(USERNAME, PASSWORD, url=URL, connect=True)
        self.session = self.client.session()
        self.curDB = None
        print('Username: {0}'.format(self.session['userCtx']['name']))
        print('Databases: {0}'.format(self.client.all_dbs()))
        self.accessDB(db_name,partition=True)
    
    def accessDB(self,db_name,partition=True):
        if(db_name in self.client.all_dbs()):
            self.curDB = self.client[db_name]
        else:
            self.curDB = self.client.create_database(db_name)
            print("creating db:",db_name)
        if self.curDB.exists():
            print('Accessing db:',db_name)
    
    def add_record(self, json_record, db_name=None, key=None):
        if(key is not None and key not in self.curDB):
            json_record['_id'] = key
            self.curDB.create_document(json_record)
        elif('id' in json_record.keys()):
            hash_string = str(hash(json_record['id']))
            if(hash_string not in self.curDB):
                json_record['_id'] = hash_string
                record = json_record
                self.curDB.create_document(record)
            else:
                print("already exist")
        elif('_id' in json_record.keys()):
            if(json_record['_id'] not in self.curDB):
                self.curDB.create_document(json_record)
    
    def delete_record(self,recordID):
        if(recordID in self.curDB):
            my_document = self.curDB[recordID]
            my_document.delete()
        elif(str(hash(recordID)) in self.curDB):
            my_document = self.curDB[str(hash(recordID))]
            my_document.delete()
        else:
            print("deletion err, record not found")
    def delete_all_records(self):
        for doc in self.curDB:
            doc.delete()
    
    def deleteDB(self,db_name):
        self.client.delete_database(db_name)
        self.curDB = None
        print(db_name,"removed")
    
    def get_data(self,n=10000):
        data = []
        count = 0
        for document in self.curDB:
            data.append(document)
            count+=1
            if(count>n):
                break
        return data
    
    def close(self):
        self.client.disconnect()
        
    def allDB(self):
        all_dbs = self.client.all_dbs()
        print('Databases: {0}'.format(all_dbs))
        return all_dbs
"""
reduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"
db2 = CloudantDB("tweets_no_keywords")

ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_zone",'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}',"_stats")
ddoc.save()

view = View(ddoc, 'view_zone')
# Assuming that 'view001' exists as part of the
# design document ddoc in the remote database...
# Use view as a callable
for row in view(limit=100,group=True)['rows']:
    print(row)
for view_name, view in ddoc.iterviews():
    print("------",view_name,view)

db2 = CloudantDB("tweets_no_keywords")
ddoc = DesignDocument(db2.curDB,'_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_time","function(doc){emit(doc.created_at.split(' ')[0], parseFloat(doc.sentiment_score))}","_stats")
ddoc.save()
view = View(ddoc, 'view_time')
for row in view(limit=100,reduce=True,group=True)['rows']:
    print(row)
"""

'\nreduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"\ndb2 = CloudantDB("tweets_no_keywords")\n\nddoc = DesignDocument(db2.curDB, \'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_zone",\'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}\',"_stats")\nddoc.save()\n\nview = View(ddoc, \'view_zone\')\n# Assuming that \'view001\' exists as part of the\n# design document ddoc in the remote database...\n# Use view as a callable\nfor row in view(limit=100,group=True)[\'rows\']:\n    print(row)\nfor view_name, view in ddoc.iterviews():\n    print("------",view_name,view)\n\ndb2 = CloudantDB("tweets_no_keywords")\nddoc = DesignDocument(db2.curDB,\'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_time","function(doc){emit(doc.created_at.split(\' \')[0], parseFloat(doc.sentiment_score))}","_stats")\nddoc.save()\nview = View(ddoc, \'view_time\')\nfor ro